# Data visualization for [CT Crime Trends](https://projects.ctmirror.org/content/2017/9/fbi-crime-data/)


In [1]:
import pandas as pd

In [2]:
import re
def fixname(name):
    #print name
    ret =  re.sub(r'[^A-Za-z ]+',"",str(name).title())
    #print name, "->", ret
    #return ret
    return ret

fixname("Test Name6")

'Test Name'

In [41]:
# Get data from sheets that include 2006 through 2016
def get_data():
    
    ret = {}
    
    ret[2006] = pd.read_excel("data/2006/06tbl08ct.xls",skiprows=4)
    ret[2007] = pd.read_excel("data/2007/07tbl08ct.xls",skiprows=4)
    ret[2008] = pd.read_excel("data/2008/08tbl08ct.xls",skiprows=4)
    ret[2009] = pd.read_excel("data/2009/09tbl08ct.xls",skiprows=4)
    ret[2010] = pd.read_excel("data/2010/10tbl08ct.xls",skiprows=4)
    ret[2011] = pd.read_excel("data/2011/Table8StateCuts/Table_8_Offenses_Known_to_Law_Enforcement_Connecticut_by_City_2011.xls",
                              skiprows=4)
    ret[2012] = pd.read_excel("data/2012/Table 8 State cuts/Table_8_Offenses_Known_to_Law_Enforcement_by_Connecticut_by_City_2012.xls",
                             skiprows=4)
    ret[2013] = pd.read_excel("data/2013/Table 8 state cuts/Table_8_Offenses_Known_to_Law_Enforcement_Connecticut_by_City_2013.xls",
                             skiprows=4)
    ret[2014] = pd.read_excel("data/2014/Table 8 pieces/Table_8_Offenses_Known_to_Law_Enforcement_by_Connecticut_by_City_2014.xls",
                             skiprows=4)
    ret[2015] = pd.read_excel("data/2015/Table 8 pieces/Table_8_Offenses_Known_to_Law_Enforcement_Connecticut_by_City_2015.xls",
                             skiprows=4)
    ret[2016] = pd.read_excel("data/2016/Table 6 pieces/Table_6_Offenses_Known_to_Law_Enforcement_Connecticut_by_City_2016.xls",
                             skiprows=4)
    ret[2017] = pd.read_excel("data/2017/Table 8 state cuts/Table_8_Offenses_Known_to_Law_Enforcement_Connecticut_by_City_2017.xls",
                             skiprows=4)

    # For some reason the FBI lists cities as states 
    # in this 2017 document.
    ret[2017]["City"] =  ret[2017]["State"]
    
    for y in range(2006,2018):
        if y == 2017:
            pass
            # For some reason the FBI lists cities as states 
            # in this 2017 document.
            #ret[y]["City"] = ret[y]["State"].apply(fixname)
        else:
            ret[y]["City"] = ret[y]["City"].apply(fixname)
    
    
    return ret
data = get_data()

In [4]:
# Make a timeline dataframe from a given column
def make_frame(col_names):
    
    df = None
    ret = None
    
    for y in range(2006,2018):
        df = None
        try:
            df = data[y]#.set_index("City")

            df["City"] = df["City"].apply(lambda x: fixname(x))
            df = df[df["City"].notnull()]
            #return df["City"]
            df = df.set_index("City")
            #return df

            df.columns = df.columns.map(lambda x: str(x).replace("\n"," "))

            for col_name in col_names:
                #print "trying", col_name, col_names
                try:
                    df[y] = df[col_name]
                    #print "got df", y, col_name
                    break
                except:
                    continue
            if ret is None:
                ret = df[[#"Population", 
                        y]]
            else:
                ret = ret.join(df[[y]],how="outer",rsuffix="_" + str(y))
        except Exception as e:
            print "Error in " + str(y), df.columns
            print "ERROR: ", e
    
    # remove all-null rows
    try:
        ret = ret.dropna(axis=0, how='all')#, thresh=None, subset=None, inplace=False)
    except:
        pass
    
    return ret

viol_crime = make_frame(["Violent crime"])
viol_crime.head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
City,,,,,,,,,,,,
Ansonia,22,38.0,26.0,36.0,27.0,30.0,29.0,33.0,36.0,25.0,24.0,22.0
Avon,5,3.0,4.0,8.0,5.0,0.0,5.0,12.0,2.0,2.0,1.0,2.0
Berlin,19,13.0,21.0,12.0,15.0,15.0,21.0,14.0,11.0,15.0,20.0,27.0
Bethel,7,6.0,4.0,1.0,5.0,1.0,6.0,3.0,0.0,4.0,1.0,9.0
Bloomfield,NaN,54.0,32.0,47.0,32.0,43.0,31.0,46.0,39.0,57.0,52.0,36.0


In [5]:
# List all column names

def list_cols():
    cols = []
    for y in range(2006,2018):
        for c in data[y].columns:
            #print c
            if c not in cols: 
                cols.append(c)
    return cols
list_cols()

[u'City',
 u'Population',
 u'Violent crime',
 u'Murder and nonnegligent manslaughter',
 u'Forcible rape',
 u'Robbery',
 u'Aggravated assault',
 u'Property crime',
 u'Burglary',
 u'Larceny-theft',
 u'Motor vehicle theft',
 u'Arson1',
 'Unnamed: 12',
 'Unnamed: 13',
 u'Arson',
 u'Violent\ncrime',
 u'Murder and\nnonnegligent\nmanslaughter',
 u'Forcible\nrape',
 u'Aggravated\nassault',
 u'Property\ncrime',
 u'Larceny-\ntheft',
 u'Motor\nvehicle\ntheft',
 u'Rape\n(revised\ndefinition)1',
 u'Rape\n(legacy\ndefinition)2',
 u'Arson3',
 u'State',
 u'Rape1']

In [6]:
# labels are file names from manual pull folder (historical data back farther than 2006)
# and the arrays are all acceptable column names used in the newer data sheets
col_labels = {
    "violent-crimes":["Violent crime",'Violent\ncrime'],
    "murder": ['Murder and nonnegligent manslaughter',
              'Murder and\nnonnegligent\nmanslaughter'],
    "rape": ['Forcible rape',
             'Forcible\nrape',
             'Rape\n(revised\ndefinition)1',
             'Rape (revised definition)1',
             'Rape\n(legacy\ndefinition)2'],
    "robbery": ['Robbery'],
    "assault": ['Aggravated assault','Aggravated\nassault'],
    'property':['Property crime','Property\ncrime'],
    'burglary':['Burglary'],
    'larceny':['Larceny-\ntheft','Larceny-theft','Larceny- theft'],
    'mv-theft':['Motor vehicle theft','Motor\nvehicle\ntheft'],
    'arson':['Arson1','Arson','Arson3']
    #'UNKNOWN':['Unnamed: 12','Unnamed: 13']
}


def make_timetable(town):

    # Dictionary of equilvant columns

    df = None
    frames = []
    for label in col_labels:
        cur_label = col_labels[label]
        #print "Trying ", cur_label
        df = make_frame(cur_label)
        df = df[df.index == town]
        #print "OK, got dataframe for " + label
        df["Crime"] = label
        frames.append(df.set_index("Crime"))
        
    return pd.concat(frames)
        
make_timetable("Hartford")

Error in 2017 Index([u'State', u'Population', u'Violent crime',
       u'Murder and nonnegligent manslaughter', u'Rape1', u'Robbery',
       u'Aggravated assault', u'Property crime', u'Burglary',
       u'Larceny- theft', u'Motor vehicle theft', u'Arson'],
      dtype='object')
ERROR:  '[2017] not in index'


/Users/jakekara/pandas/lib/python2.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Crime,,,,,,,,,,,,
larceny,5957,4437.0,4048.0,4061.0,3639.0,3372.0,3467.0,3416.0,3642.0,3613.0,3616.0,3490.0
assault,761,680.0,782.0,921.0,982.0,955.0,965.0,840.0,814.0,835.0,798.0,804.0
murder,24,31.0,31.0,33.0,26.0,27.0,23.0,23.0,19.0,32.0,14.0,29.0
arson,87,107.0,97.0,84.0,95.0,91.0,88.0,45.0,45.0,55.0,112.0,87.0
robbery,758,642.0,629.0,600.0,570.0,602.0,640.0,557.0,502.0,510.0,476.0,465.0
mv-theft,1709,1601.0,1326.0,993.0,837.0,859.0,802.0,639.0,750.0,897.0,773.0,679.0
burglary,1158,1168.0,1000.0,1135.0,1019.0,1271.0,1050.0,981.0,888.0,958.0,808.0,738.0
property,8824,7206.0,6374.0,6189.0,5495.0,5502.0,5319.0,5036.0,5280.0,5468.0,5197.0,4907.0
violent-crimes,1590,1411.0,1503.0,1603.0,1624.0,1639.0,1655.0,1473.0,1380.0,1421.0,1353.0,1343.0


In [7]:
make_timetable("Bethel")

Error in 2017 Index([u'State', u'Population', u'Violent crime',
       u'Murder and nonnegligent manslaughter', u'Rape1', u'Robbery',
       u'Aggravated assault', u'Property crime', u'Burglary',
       u'Larceny- theft', u'Motor vehicle theft', u'Arson'],
      dtype='object')
ERROR:  '[2017] not in index'


/Users/jakekara/pandas/lib/python2.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Crime,,,,,,,,,,,,
larceny,93,84.0,128.0,171.0,158.0,253.0,263.0,143.0,135.0,141.0,121.0,147.0
assault,3,3.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0
murder,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
arson,1,1.0,0.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0
robbery,3,1.0,3.0,1.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,3.0
mv-theft,6,2.0,9.0,7.0,6.0,6.0,10.0,6.0,7.0,2.0,15.0,4.0
burglary,22,22.0,30.0,36.0,31.0,47.0,48.0,31.0,22.0,26.0,26.0,20.0
property,121,108.0,167.0,214.0,195.0,306.0,321.0,180.0,164.0,169.0,162.0,171.0
violent-crimes,7,6.0,4.0,1.0,5.0,1.0,6.0,3.0,0.0,4.0,1.0,9.0


In [8]:
# generate a list of all cities

# def all_cities():
#     ret = []
#     for y in range(2006,2017):
#         df = get_data()[y].set_index("City").dropna(axis=0,how='any')
#         cities = df.index
#         for c in cities:
#             if c in ret: 
#                 continue
#             ret.append(c)
#     return ret

all_cities = get_data()[2016]["City"].unique().tolist()

all_cities[:10]
    

['Ansonia',
 'Avon',
 'Berlin',
 'Bethel',
 'Bloomfield',
 'Branford',
 'Bridgeport',
 'Bristol',
 'Brookfield',
 'Canton']

In [28]:
len(all_cities)

93

In [9]:
# make a timetable for each city and save it to a file
def make_all_timetables():
    for c in all_cities:
        make_timetable(c).to_csv("output/" + c + ".csv")
#make_all_timetables()

# Adding in historical data 

The manual_pull folder contains files with years older than 2006. I want to combine them with the newer sheets and remove conflicting values

In [10]:
import os

def get_csvs(folder = "data/manual_pull/",skiprows=0):
    ret = {}
    
    for fname in os.listdir(folder):
        fpath = os.path.join(folder, fname)
        topic = fname.replace(".csv","")
        df = pd.read_csv(fpath,skiprows=skiprows)
        df["City"] = df["Agency"].str.replace(" Town Police Dept","")
        df["City"] = df["City"].str.replace(" Police Dept", "")
        df["City"] = df["City"].str.replace(" Police", "")
        
        # combine complementary columns based on def'n change
#         if topic == "rape": 
#             df = df.groupby(df.index).max()
#             #print df
            
        ret[topic] = df.set_index("City").dropna(thresh=5)

    return ret

hist_dfs = get_csvs(skiprows=4)

# df = hist_dfs["rape"]
# hist_dfs["rape"] = df.groupby(df.index).max()

# df = hist_dfs["violent-crimes"]
# hist_dfs["violent-crimes"] = df.groupby(df.index).max()


In [31]:
len(hist_dfs["violent-crimes"])

83

In [29]:
# Check for duplicates
hist_dfs["violent-crimes"][hist_dfs["violent-crimes"].index.duplicated()]

,Agency,State,1985,1986,1987,1988,1989,1990,1991,1992,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,Unnamed: 32
City,,,,,,,,,,,,,,,,,,,,,


In [64]:
# Combine the history dfs with the newer dfs
def add_history(df, hist_df):
    
    new = df.copy()

    new = new.rename(columns = lambda x: str(x))
    print new.columns
    
#     ret = hist_df.join(new,how="left",lsuffix="_hist",rsuffix="_new")
    ret = hist_df.merge(
        new,
        #2018
        how="outer",
        suffixes=["_hist","_new"],
        left_index=True,right_index=True)

#     ret = hist_df.reset_index()\
#     .merge(new.reset_index(),on="City",how="left",suffixes=["_hist","_new"],left_index=True,right_index=True)
#     ret = ret.set_index("City")
    
    print ret.columns
    return ret
    ret = ret[ret["Agency"].notnull()]
    return ret.dropna(axis=0,how="all")
    
murder_historical = add_history(make_frame(col_labels["murder"]),hist_dfs["murder"])#.columns
#murder_historical[murder_historical.index.duplicated(keep=False)]
murder_historical[murder_historical.index=="Hartford"]

Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016', u'2017'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991', u'1992', u'1993', u'1994', u'1995', u'1996', u'1997',
       u'1998', u'1999', u'2000', u'2001', u'2002', u'2003', u'2004', u'2005',
       u'2006_hist', u'2007_hist', u'2008_hist', u'2009_hist', u'2010_hist',
       u'2011_hist', u'2012_hist', u'2013_hist', u'2014_hist', u'Unnamed: 32',
       u'2006_new', u'2007_new', u'2008_new', u'2009_new', u'2010_new',
       u'2011_new', u'2012_new', u'2013_new', u'2014_new', u'2015', u'2016',
       u'2017'],
      dtype='object')


,Agency,State,1985,1986,1987,1988,1989,1990,1991,1992,...,2008_new,2009_new,2010_new,2011_new,2012_new,2013_new,2014_new,2015,2016,2017
City,,,,,,,,,,,,,,,,,,,,,
Hartford,Hartford Police Dept,CT,12.0,28.0,24.0,37.0,30.0,19.0,24.0,13.0,...,31.0,33.0,26.0,27.0,23.0,23.0,19.0,32.0,14.0,29.0


In [65]:
new_hartford_murders = make_frame(col_labels["murder"])[make_frame(col_labels["murder"]).index == "Hartford"]
hist_hartford_murders = hist_dfs["murder"][hist_dfs["murder"].index=="Hartford"]
new_hartford_murders = new_hartford_murders.rename(columns = lambda x: str(x))
new_hartford_murders.columns
new_hartford_murders.merge(hist_hartford_murders,
                           how="outer",
                           suffixes=["_hist","_new"],
                           left_index=True,right_index=True).transpose()#.columns

City,Hartford
2006_hist,24
2007_hist,31
2008_hist,31
2009_hist,33
2010_hist,26
2011_hist,27
2012_hist,23
2013_hist,23
2014_hist,19
2015,32


In [66]:
# Check columns that overlap to see if the historical data has the same valus as the newer sheets
err_count = 0
tot_count = 0
def add_checks(df):
    
    global err_count, tot_count
    
    err_count = 0
    tot_count = 0
    
    ret = df.copy()
    overlap = map(lambda x: x.replace("_new",""),
                  filter(lambda x: "_new" in x, ret.columns.get_values()))

    for y in overlap:
        #print ret.dtypes
        ret[y + "_new"] = pd.to_numeric(ret[y + "_new"],errors=coerce)
        ret[y + "_hist"] = pd.to_numeric(ret[y + "_hist"],errors=coerce)
        ret[y + "_diff"] = ret[y + "_new"].sub(ret[y + "_hist"])
        ret[y + "_diff_pct"] = ret[y + "_diff"] / ret[y + "_hist"]
        def y_or_null(row,y):
            global tot_count,err_count
            tot_count += 1
            if row[y + "_new"] == row[y + "_hist"]:
                return row[y + "_new"]
            #print row[y + "_new"] - row[y + "_hist"]
            err_count += 1
            return None
        ret[y] = ret.apply(lambda x: y_or_null(x,y) ,axis=1)
        
    print "Errors: " + str(err_count)
    print "Total:  " + str(tot_count)
    return ret
    
add_checks(add_history(make_frame(col_labels["murder"]),
                       hist_dfs["murder"]))#.head(1).transpose()

Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016', u'2017'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991', u'1992', u'1993', u'1994', u'1995', u'1996', u'1997',
       u'1998', u'1999', u'2000', u'2001', u'2002', u'2003', u'2004', u'2005',
       u'2006_hist', u'2007_hist', u'2008_hist', u'2009_hist', u'2010_hist',
       u'2011_hist', u'2012_hist', u'2013_hist', u'2014_hist', u'Unnamed: 32',
       u'2006_new', u'2007_new', u'2008_new', u'2009_new', u'2010_new',
       u'2011_new', u'2012_new', u'2013_new', u'2014_new', u'2015', u'2016',
       u'2017'],
      dtype='object')
Errors: 135
Total:  837


,Agency,State,1985,1986,1987,1988,1989,1990,1991,1992,...,2011,2012_diff,2012_diff_pct,2012,2013_diff,2013_diff_pct,2013,2014_diff,2014_diff_pct,2014
City,,,,,,,,,,,,,,,,,,,,,
Ansonia,Ansonia Police Dept,CT,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,3.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
Avon,Avon Police Dept,CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
Berlin,Berlin Police Dept,CT,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
Bethel,Bethel Police Dept,CT,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
Bloomfield,Bloomfield Police Dept,CT,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0
Branford,Branford Police Dept,CT,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,NaN,0.0,-1.0,-1.0,NaN,0.0,0.0,1.0
Bridgeport,Bridgeport Police Dept,CT,30.0,40.0,37.0,43.0,36.0,57.0,51.0,57.0,...,20.0,0.0,0.0,22.0,0.0,0.0,11.0,0.0,0.0,11.0
Bristol,Bristol Police Dept,CT,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,3.0
Brookfield,Brookfield Police Dept,CT,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0


In [67]:
def make_hist_table(topic):
    
    try:

        return add_checks(add_history(make_frame(col_labels[topic]),
                                      hist_dfs[topic]))
    except:
        # fall back if can't get history data. arson doesn't have it
        return make_frame(col_labels[topic])
    

#make_hist_table("rape")

add_history(make_frame(col_labels["rape"]),
                       hist_dfs["rape"])#.index.unique()

Error in 2017 Index([u'State', u'Population', u'Violent crime',
       u'Murder and nonnegligent manslaughter', u'Rape1', u'Robbery',
       u'Aggravated assault', u'Property crime', u'Burglary',
       u'Larceny- theft', u'Motor vehicle theft', u'Arson'],
      dtype='object')
ERROR:  '[2017] not in index'
Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991', u'1992', u'1993', u'1994', u'1995', u'1996', u'1997',
       u'1998', u'1999', u'2000', u'2001', u'2002', u'2003', u'2004', u'2005',
       u'2006_hist', u'2007_hist', u'2008_hist', u'2009_hist', u'2010_hist',
       u'2011_hist', u'2012_hist', u'2013_hist', u'2014_hist', u'Unnamed: 32',
       u'2006_new', u'2007_new', u'2008_new', u'2009_new', u'2010_new',
       u'2011_new', u'2012_new', u'2013_new', u'2014_new', u'2015', u'2016'],
      dtype='objec

,Agency,State,1985,1986,1987,1988,1989,1990,1991,1992,...,2007_new,2008_new,2009_new,2010_new,2011_new,2012_new,2013_new,2014_new,2015,2016
City,,,,,,,,,,,,,,,,,,,,,
Agency,Agency,State,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ansonia,Ansonia Police Dept,CT,33.0,5.0,1.0,5.0,4.0,1.0,10.0,3.0,...,3.0,3.0,4.0,2.0,3.0,4.0,5.0,6.0,5.0,2.0
Avon,Avon Police Dept,CT,0.0,0.0,2.0,0.0,3.0,2.0,1.0,0.0,...,0.0,1.0,3.0,1.0,0.0,2.0,8.0,1.0,1.0,0.0
Berlin,Berlin Police Dept,CT,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,1.0,2.0,1.0,0.0,5.0,3.0,2.0,3.0
Bethel,Bethel Police Dept,CT,3.0,4.0,4.0,3.0,3.0,4.0,11.0,1.0,...,2.0,0.0,0.0,1.0,0.0,3.0,2.0,0.0,4.0,0.0
Bloomfield,Bloomfield Police Dept,CT,3.0,5.0,5.0,6.0,2.0,6.0,3.0,4.0,...,9.0,4.0,12.0,3.0,6.0,5.0,9.0,6.0,8.0,9.0
Branford,Branford Police Dept,CT,0.0,2.0,3.0,0.0,4.0,1.0,4.0,4.0,...,6.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Bridgeport,Bridgeport Police Dept,CT,57.0,75.0,58.0,90.0,70.0,79.0,82.0,74.0,...,77.0,58.0,58.0,57.0,116.0,388.0,NaN,92.0,80.0,69.0
Bristol,Bristol Police Dept,CT,4.0,2.0,6.0,3.0,5.0,12.0,6.0,2.0,...,12.0,27.0,26.0,27.0,10.0,9.0,15.0,11.0,11.0,10.0


In [68]:
def make_all_tables_with_hist():
    ret = {}
    for label in col_labels:
        print label
        try:
            ret[label] = make_hist_table(label)
            ret[label].to_csv("output/historical-" + label + ".csv" )
        except Exception as e:
            print e
            continue
            
    return ret
        
all_hist_tables = make_all_tables_with_hist()

larceny
Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016', u'2017'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991', u'1992', u'1993', u'1994', u'1995', u'1996', u'1997',
       u'1998', u'1999', u'2000', u'2001', u'2002', u'2003', u'2004', u'2005',
       u'2006_hist', u'2007_hist', u'2008_hist', u'2009_hist', u'2010_hist',
       u'2011_hist', u'2012_hist', u'2013_hist', u'2014_hist', u'Unnamed: 32',
       u'2006_new', u'2007_new', u'2008_new', u'2009_new', u'2010_new',
       u'2011_new', u'2012_new', u'2013_new', u'2014_new', u'2015', u'2016',
       u'2017'],
      dtype='object')
Errors: 405
Total:  837
assault
Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016', u'2017'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991

In [51]:
len(all_hist_tables["arson"])

93

In [70]:
len(all_hist_tables["murder"])

93

In [79]:
def make_mega_historical():
    tmp_mega_hist = make_all_tables_with_hist()
    df = None
    frames = []
    for topic in tmp_mega_hist:
        df = tmp_mega_hist[topic].copy()
        df["crime"] = topic
        frames.append(df)
        
    return pd.concat(frames).dropna(thresh=5)


def exclude(name):
    # 2018 me can't figure out why 2017 me did this.
    
    return name not in ["Agency",
#                         "Groton Long Point",
#                         "Groton Town",
#                         "Middlebury",
#                         "Middlebury",
#                         "Portland",
#                         "Porttland",
#                         "Putnam",
#                         "Easton",
#                         "Redding",
#                         "Thomaston",
#                         "Woodbridge"
                       ]

#print mega_hist.index.map(lambda x: exclude(x))

mega_hist = make_mega_historical()
mega_hist = mega_hist[mega_hist.index.map(exclude)]
mega_hist[mega_hist.index == "Easton"]

#mega_hist = mega_hist[mega_hist.index.map(lambda x: exclude(x))]
mega_hist.to_csv("output/historical-all.csv")
mega_hist.to_csv("viz/data/historical-all.csv")

#print mega_hist["crime"].unique()

larceny
Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016', u'2017'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991', u'1992', u'1993', u'1994', u'1995', u'1996', u'1997',
       u'1998', u'1999', u'2000', u'2001', u'2002', u'2003', u'2004', u'2005',
       u'2006_hist', u'2007_hist', u'2008_hist', u'2009_hist', u'2010_hist',
       u'2011_hist', u'2012_hist', u'2013_hist', u'2014_hist', u'Unnamed: 32',
       u'2006_new', u'2007_new', u'2008_new', u'2009_new', u'2010_new',
       u'2011_new', u'2012_new', u'2013_new', u'2014_new', u'2015', u'2016',
       u'2017'],
      dtype='object')
Errors: 405
Total:  837
assault
Index([u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013',
       u'2014', u'2015', u'2016', u'2017'],
      dtype='object')
Index([u'Agency', u'State', u'1985', u'1986', u'1987', u'1988', u'1989',
       u'1990', u'1991

/Users/jakekara/pandas/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [69]:
len(mega_hist.index.unique())

93

In [54]:
mega_hist

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,...,2014_diff_pct,2014_hist,2014_new,2015,2016,2017,Agency,State,Unnamed: 32,crime
City,,,,,,,,,,,,,,,,,,,,,
Ansonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,Ansonia Police Dept,CT,NaN,murder
Avon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,Avon Police Dept,CT,NaN,murder
Berlin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,Berlin Police Dept,CT,NaN,murder
Bethel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,1.0,0.0,Bethel Police Dept,CT,NaN,murder
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,1.0,0.0,0.0,Bloomfield Police Dept,CT,NaN,murder
Branford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,0.0,Branford Police Dept,CT,NaN,murder
Bridgeport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,11.0,11.0,19.0,10.0,21.0,Bridgeport Police Dept,CT,NaN,murder
Bristol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3.0,3.0,0.0,0.0,1.0,Bristol Police Dept,CT,NaN,murder
Brookfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,Brookfield Police Dept,CT,NaN,murder


In [55]:
mega_hist[mega_hist.index == "Wolcott"]

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,...,2014_diff_pct,2014_hist,2014_new,2015,2016,2017,Agency,State,Unnamed: 32,crime
City,,,,,,,,,,,,,,,,,,,,,
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,Wolcott Police Dept,CT,NaN,murder
Wolcott,1.0,3.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arson
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,18.0,18.0,27.0,38.0,28.0,Wolcott Police Dept,CT,NaN,mv-theft
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,7.0,7.0,5.0,5.0,5.0,Wolcott Police Dept,CT,NaN,violent-crimes
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,245.0,245.0,178.0,185.0,183.0,Wolcott Police Dept,CT,NaN,larceny
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3.0,3.0,2.0,1.0,1.0,Wolcott Police Dept,CT,NaN,assault
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,4.0,4.0,3.0,4.0,2.0,Wolcott Police Dept,CT,NaN,robbery
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,44.0,44.0,29.0,65.0,67.0,Wolcott Police Dept,CT,NaN,burglary
Wolcott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,307.0,307.0,234.0,288.0,278.0,Wolcott Police Dept,CT,NaN,property


In [56]:
len(mega_hist)

840

In [57]:
mega_hist[mega_hist.index == "Bloomfield"]

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,...,2014_diff_pct,2014_hist,2014_new,2015,2016,2017,Agency,State,Unnamed: 32,crime
City,,,,,,,,,,,,,,,,,,,,,
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,1.0,0.0,0.0,Bloomfield Police Dept,CT,NaN,murder
Bloomfield,NaN,3.0,1.0,1.0,2.0,3.0,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arson
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,22.0,22.0,22.0,43.0,66.0,Bloomfield Police Dept,CT,NaN,mv-theft
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.025000,40.0,39.0,57.0,52.0,36.0,Bloomfield Police Dept,CT,NaN,violent-crimes
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,402.0,402.0,417.0,418.0,461.0,Bloomfield Police Dept,CT,NaN,larceny
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,21.0,21.0,42.0,32.0,16.0,Bloomfield Police Dept,CT,NaN,assault
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,12.0,12.0,6.0,11.0,10.0,Bloomfield Police Dept,CT,NaN,robbery
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.122222,90.0,79.0,66.0,54.0,48.0,Bloomfield Police Dept,CT,NaN,burglary
Bloomfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.021401,514.0,503.0,505.0,515.0,575.0,Bloomfield Police Dept,CT,NaN,property


In [72]:
import numpy as np

import matplotlib
%matplotlib inline
matplotlib.style.use('ggplot')

import matplotlib.pyplot as plt



csfont = {'fontname':'Monaco'}
hfont = {'fontname':'Helvetica'}


def small(city, crime):
    
    ret = mega_hist[(mega_hist.index == city) &
              (mega_hist["crime"] == crime)]

    cols = filter(
        lambda x: x in ret.columns,
        map(lambda x: str(x), range(1985,2018)))

    #print cols
    
    ret.columns = ret.columns.map(lambda x: str(x))
    ret = ret[cols]
    
    
    #return ret
#     for col in cols:
#         ret[col] = pd.to_numeric(ret[col],errors=coerce)
    #print ret
    ret = ret.transpose()

    ret = ret.reset_index()
    ret.columns = ["year","count"]
    ret["year"] = pd.to_numeric(ret["year"],errors=coerce)
    
    
    fig = plt.figure()   

    fig.set_figheight(1)
    fig.set_figwidth(4)
    #ax = fig.add_subplot(111)
    

    x = np.array(pd.to_numeric(ret["year"],errors=coerce).values)
    ret["count"] = pd.to_numeric(ret["count"],errors=coerce)
    y = np.array(pd.to_numeric(ret["count"],errors=coerce).values)

    ax = ret.plot.scatter(x="year", y="count", figsize=(2.7,1.5))
    idx = np.isfinite(x) & np.isfinite(y)
    x = x[idx]
    y = y[idx]

    fit = np.polyfit(x,y,1) 
    
    print fit
    ax.plot(x, fit[0] * x + fit[1], color='orange')

    ax.set_xticks([1987,1997,2007,2017])
    
    ax.set_yticks([0,ret["count"].max()])
    ax.set_xlabel("")
    ax.set_ylabel("")
    #ax.set_title(crime.replace("-"," ") + ", " + city,**csfont)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    plt.tight_layout()
              



    plt.savefig("output/figures/" + city + "-" + crime + ".png")
    plt.savefig("viz/img/" + city + "-" + crime + ".png")

    plt.close()

    
#ax = small("Bethel","larceny").plot.scatter(x="year", y="count", figsize=(3,1))

#plt.show()

small("Bridgeport","mv-theft")

[-1.17432152e+02  2.37068131e+05]


<Figure size 288x72 with 0 Axes>

In [73]:
small("Hartford","murder")

[-1.53743316e-02  5.67034314e+01]


<Figure size 288x72 with 0 Axes>

In [74]:
small("Naugatuck","violent-crimes")

[-1.02685973e-01  2.40792224e+02]


<Figure size 288x72 with 0 Axes>

In [75]:
def make_all_plots(crimes = col_labels):
    for city in mega_hist.reset_index()["City"].unique():
        for crime in crimes:
            try:
                small(city, crime)
            except:
                print city, crime
                continue
#make_all_plots(crimes=["violent-crimes"])
make_all_plots()

[-1.91966523e+00  4.13865221e+03]
[-1.25251761e+00  2.53662108e+03]
[ 4.73793308e-03 -8.97971572e+00]
[-3.49650350e-02  7.14988345e+01]
[-5.17379679e-01  1.05285250e+03]
[-7.04882081e-01  1.46283183e+03]
[-3.37303855e+00  6.82589979e+03]
[-5.81417830e+00  1.20629456e+04]
[-2.01656330e+00  4.08899492e+03]
[-2.40930798e-01  4.87651396e+02]
[-3.59458560e+00  7.36564924e+03]
[-1.54151132e-01  3.10819310e+02]
[ 6.68449198e-04 -1.30726381e+00]
[ 1.74825175e-02 -3.49160839e+01]
[-2.47326203e-02  5.13384581e+01]
[-2.32954545e-01  4.73626894e+02]
[-2.15240642e+00  4.34666221e+03]
[-6.03622168e+00  1.22981524e+04]
[-1.45007575e-01  2.95369374e+02]
[ 5.09794533e-03 -9.46068808e+00]
[ 1.44837103e+00 -2.62280957e+03]
[ 3.51296671e-01 -6.97175832e+02]
[ 3.34224599e-04 -3.96056150e-01]
[-1.04895105e-01  2.12496503e+02]
[ 1.04297982e-01 -2.04149456e+02]
[-7.81082888e-01  1.59582565e+03]
[-8.05854356e-01  1.68229960e+03]
[-6.55627356e-02  5.09308966e+02]
[ 5.28600331e-01 -1.04584992e+03]
[ 4.15158642e-

[-4.30113636e+00  8.69911932e+03]
[-8.88877120e+00  1.85210831e+04]
[-1.73294442e-01  3.65141120e+02]
[ 2.09362218e-02 -3.93752462e+01]
[-8.82897301e+00  1.80325163e+04]
[-5.24349234e-02  1.14116453e+02]
[-9.31849319e-03  1.88606868e+01]
[-1.29323308e-01  2.60717293e+02]
[ 8.06734678e-02 -1.55546043e+02]
[-7.19369558e-01  1.45955878e+03]
[-3.04828623e+00  6.18434733e+03]
[-1.17346515e+01  2.39597497e+04]
[-1.12347321e-02  3.91816786e+01]
[ 1.18980582e-02 -2.19589653e+01]
[-6.78712281e-01  1.46427946e+03]
[-8.32219251e-02  1.69193739e+02]
[0. 0.]
[-1.11888112e-01  2.25729604e+02]
[-4.01069519e-03  8.60115865e+00]
[-4.49197861e-01  9.08360071e+02]
[-7.75936346e-01  1.57482640e+03]
[-1.89719573e+00  3.93418373e+03]
[-5.11363636e-02  1.06657197e+02]
[ 4.90501204e-02 -9.69822558e+01]
[-3.90490038e+01  7.88424435e+04]
[-4.55870316e-01  9.30204574e+02]
[ 4.42066077e-03 -8.50116333e+00]
[-2.09790210e-01  4.23993007e+02]
[-3.81590666e-01  7.75273869e+02]
[-4.73873820e+00  9.56801544e+03]
[-7.51

[-2.16074750e+00  4.53435152e+03]
[-2.94283256e-03  1.27942142e+01]
[ 4.23044966e-02 -8.41268149e+01]
[-5.35197989e+00  1.12007964e+04]
[-5.15564608e-01  1.04228090e+03]
[-2.94988358e-03  6.15225613e+00]
[-1.72727273e-01  3.48163636e+02]
[-2.15369105e-01  4.41985348e+02]
[-2.20383329e+00  4.46988829e+03]
[-3.19743238e+00  6.49958252e+03]
[-1.07080201e+01  2.20801190e+04]
[-7.29490846e-01  1.48248797e+03]
[-1.59582357e-02  3.26657373e+01]
[-7.09649570e+01  1.44176608e+05]
[ 3.65821696e+00 -7.15595501e+03]
[-1.11979962e-01  2.27478588e+02]
[-2.63636364e-01  5.36163636e+02]
[-3.84019861e+00  7.80869153e+03]
[-1.81152823e+01  3.66040286e+04]
[-4.30598371e+01  8.68215536e+04]
[-1.31782179e+02  2.66889897e+05]
[-2.05590035e-01  7.14376676e+02]
[ 8.27407265e-02 -1.53154622e+02]
[-2.16131657e+01  4.40648750e+04]
[-3.07471865e-01  7.01888236e+02]
[-7.85145657e-03  1.70635771e+01]
[-5.83916084e-01  1.18529720e+03]
[-1.58578421e-01  3.59399260e+02]
[-2.37058110e+00  4.82788462e+03]
[-6.16292934e+

[-1.66517804e+01  3.43220133e+04]
[ 3.49580478e-01 -6.75318939e+02]
[ 1.51971685e-01 -3.02264196e+02]
[-1.53393763e+01  3.11372658e+04]
[-1.43992770e+00  2.90865264e+03]
[ 2.39151276e-03 -4.56629337e+00]
[-5.54545455e-01  1.11892727e+03]
[-2.44427417e-01  5.03852605e+02]
[-1.33598121e+00  2.71952714e+03]
[-5.90931031e+00  1.19489647e+04]
[-2.29155932e+01  4.64644162e+04]
[-1.51928680e+00  3.08720954e+03]
[ 6.60724350e-02 -1.28657862e+02]
[-1.22419218e+01  2.52290461e+04]
[-2.09779215e-01  4.36590230e+02]
[-1.00267380e-02  2.05180481e+01]
[-4.30069930e-01  8.67168998e+02]
[-8.99064171e-02  1.90387589e+02]
[-4.94786096e+00  9.98115463e+03]
[-6.79397800e+00  1.37753412e+04]
[-2.36585884e+01  4.83385553e+04]
[-3.41111730e-01  7.14393239e+02]
[-3.64103023e-03  1.15901453e+01]
[-5.90795455e+01  1.22440473e+05]
[-2.96724599e+00  6.16803498e+03]
[-9.05748663e-02  1.88573641e+02]
[ 9.12587413e-01 -1.82758625e+03]
[-2.99799465e+00  6.24465397e+03]
[-1.66590909e+01  3.41704167e+04]
[-6.24719251e+

<Figure size 288x72 with 0 Axes>

...

<Figure size 288x72 with 0 Axes>

<Figure size 288x72 with 0 Axes>

<Figure size 288x72 with 0 Axes>

<Figure size 288x72 with 0 Axes>

<Figure size 288x72 with 0 Axes>

In [76]:
def debug_chart(city, crime):

    return mega_hist[(mega_hist.index == city) &
              (mega_hist["crime"] == crime)]

debug_chart("Easton","larceny")

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,...,2014_diff_pct,2014_hist,2014_new,2015,2016,2017,Agency,State,Unnamed: 32,crime
City,,,,,,,,,,,,,,,,,,,,,
Easton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20.0,16.0,33.0,6.0,NaN,NaN,NaN,larceny


In [77]:
debug_chart("Hartford","murder")

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,...,2014_diff_pct,2014_hist,2014_new,2015,2016,2017,Agency,State,Unnamed: 32,crime
City,,,,,,,,,,,,,,,,,,,,,
Hartford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,19.0,19.0,32.0,14.0,29.0,Hartford Police Dept,CT,NaN,murder
